In [2]:
import os

import keras
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, MaxPooling2D)
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
num_classes = 7
img_rows,img_cols = 48 , 48
batch_size = 2

train_data_dir = r"D:\LEARNING\AI\FACE EXPRESSIOON\images\train" #Train images path here
validation_data_dir = r"D:\LEARNING\AI\FACE EXPRESSIOON\images\validation" #Test images path here

train_data_gen = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 30,
                                    shear_range = 0.3,
                                    zoom_range = 3.0,
                                    width_shift_range = 0.4,
                                    height_shift_range = 0.4,
                                    horizontal_flip = True)
"""
since my dataset not enough (too small) for training I'm making it different images from this by rotating
the image by 30% and changing the shear angle and zooming range also shifting the wight and height while
retaining the dimensions and also by producing mirror images.
"""
validation_data_gen = ImageDataGenerator(rescale = 1./255)

train_generator = train_data_gen.flow_from_directory(train_data_dir,
                                                    color_mode = "grayscale",
                                                    target_size = (img_cols,img_rows)
                                                    ,batch_size = batch_size,
                                                    class_mode = "categorical",
                                                    shuffle = True)

validation_generator = validation_data_gen.flow_from_directory(validation_data_dir,
                                                                color_mode = "grayscale",
                                                                target_size = (img_cols,img_rows),
                                                                batch_size = batch_size,
                                                                class_mode = "categorical",
                                                                shuffle = True)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [3]:
model = Sequential() # using sequential modeling

#block1
model.add(Conv2D(32,(3,3),padding="same",
                 kernel_initializer ="he_normal",input_shape=(img_rows,img_cols,1)))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(32,(3,3),padding="same",
                 kernel_initializer ="he_normal",input_shape=(img_rows,img_cols,1)))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [4]:
#block2
model.add(Conv2D(64,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(64,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [5]:
#block3
model.add(Conv2D(128,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(128,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [6]:
#block4
model.add(Conv2D(256,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(256,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [7]:
#block5
model.add(Conv2D(512,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(512,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [8]:
#block6
model.add(Flatten())
model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [9]:
#block7
model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [10]:
model.add(Dense(num_classes,kernel_initializer="he_normal"))
model.add(Activation("softmax"))

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [12]:
from keras.optimizers import RMSprop , SGD , Adam
from keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau

In [13]:
checkpoint = ModelCheckpoint("checkpoint2.h5",
                        monitor="val_loss",
                        mode="min",
                        save_best_only=True,
                        verbose=1) # checkpoint file name here in my case checkpoint2.h5

"""
Here we will monitor the val_loss which is minimum and saving it
"""

earlystop = EarlyStopping(monitor="val_loss",
                        min_delta=0,
                        patience=3,
                        restore_best_weights=True,
                        verbose=1)

"""
Here we will monitor the val_loss whith 3 extra epoches and retain the best weighted file
"""

reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

In [14]:
callbacks=[earlystop,checkpoint,reduce_lr]

model.compile(loss="categorical_crossentropy",
             optimizer=Adam(lr=0.001),
             metrics=["accuracy"] )

nb_train_samples = 28821 #obtained after train and validation generators
nb_validation_samples = 7066 #obtained after train and validation generators
epochs = 25

#fitting/training our model

history = model.fit(train_generator,
                             steps_per_epoch=nb_train_samples//batch_size,
                             epochs=epochs,
                             callbacks=callbacks,
                             validation_data=validation_generator,
                             validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
14409/14410 [============================>.] - ETA: 0s - loss: 1.9104 - accuracy: 0.2183
Epoch 00001: val_loss improved from inf to 1.85782, saving model to checkpoint2.h5
14410/14410 [==============================] - 1091s 76ms/step - loss: 1.9104 - accuracy: 0.2183 - val_loss: 1.8578 - val_accuracy: 0.2498
Epoch 2/25
14409/14410 [============================>.] - ETA: 0s - loss: 1.8183 - accuracy: 0.2442
Epoch 00002: val_loss improved from 1.85782 to 1.81559, saving model to checkpoint2.h5
14410/14410 [==============================] - 322s 22ms/step - loss: 1.8184 - accuracy: 0.2441 - val_loss: 1.8156 - val_accuracy: 0.2536
Epoch 3/25
14410/14410 [==============================] - ETA: 0s - loss: 1.8120 - accuracy: 0.2478
Epoch 00003: val_loss improved from 1.81559 to 1.79990, saving model to checkpoint2.h5
14410/14410 [==============================] - 249s 17ms/step - loss: 1.8120 - accuracy: 0.2478 - val_loss: 1.7999 - val_accuracy: 0.2638
Epoch 4/25
14408/14410 [====